In [28]:

import gensim
import smart_open
import collections
import pandas as pd

In [14]:
file=pd.read_csv('Competition_Train_Data.txt', sep='\t')

In [15]:
def read_corpus(fname, tokens_only=False):
    with smart_open.smart_open(fname, encoding="iso-8859-1") as f:
        for i, line in enumerate(f):
            if tokens_only:
                yield gensim.utils.simple_preprocess(line)
            else:
                # For training data, add tags
                yield gensim.models.doc2vec.TaggedDocument(gensim.utils.simple_preprocess(line), [i])

In [19]:
test_corpus = list(read_corpus('/Users/HermanCheung/Documents/data_comp/Competition_Test_Data.txt', tokens_only=True))
train_corpus = list(read_corpus('/Users/HermanCheung/Documents/data_comp/Competition_Train_Data.txt'))

In [33]:
print(test_corpus[:2])


[['unique', 'id', 'string', 'string'], ['amrozi', 'accused', 'his', 'brother', 'whom', 'he', 'called', 'the', 'witness', 'of', 'deliberately', 'distorting', 'his', 'evidence', 'referring', 'to', 'him', 'as', 'only', 'the', 'witness', 'amrozi', 'accused', 'his', 'brother', 'of', 'deliberately', 'distorting', 'his', 'evidence']]


In [34]:
train_corpus[:2]


[TaggedDocument(words=['label', 'unique', 'id', 'string', 'string'], tags=[0]),
 TaggedDocument(words=['paul', 'themba', 'nyathi', 'an', 'mdc', 'spokesman', 'said', 'the', 'avenues', 'clinic', 'was', 'being', 'targeted', 'because', 'it', 'was', 'treating', 'opposition', 'supporters', 'mdc', 'spokesperson', 'paul', 'themba', 'nyathi', 'said', 'the', 'avenues', 'clinic', 'was', 'targeted', 'because', 'it', 'was', 'handling', 'opposition', 'supporters'], tags=[1])]

In [35]:
model = gensim.models.doc2vec.Doc2Vec(vector_size=150, min_count=2, epochs=40)


In [36]:
model.build_vocab(train_corpus)

In [37]:
%time model.train(train_corpus, total_examples=model.corpus_count, epochs=model.epochs)


CPU times: user 32.1 s, sys: 4.78 s, total: 36.9 s
Wall time: 22.3 s


In [38]:
model.infer_vector(['only', 'you', 'can', 'prevent', 'forest', 'fires'])


array([ 0.10399696, -0.13810669, -0.01119446, -0.2727221 ,  0.09268144,
       -0.08021732,  0.0923015 , -0.07271127, -0.22452857, -0.09475417,
        0.0272525 , -0.28034917, -0.12716474,  0.1927103 , -0.03581752,
        0.07468475, -0.07126505,  0.13248113,  0.10620719,  0.30943698,
        0.06757701, -0.13212407,  0.24632797, -0.01693263,  0.01812239,
        0.10849672, -0.19573966,  0.09077448,  0.17788196,  0.25805816,
       -0.09968322,  0.00255157,  0.22304344, -0.01869166, -0.11780396,
       -0.03459014, -0.17984794, -0.07049344,  0.05687457, -0.2342503 ,
       -0.07182845,  0.13460226,  0.03835825,  0.1572238 ,  0.1871595 ,
       -0.04727624,  0.04894815, -0.2528191 ,  0.18194388,  0.02861961,
       -0.08772141, -0.09115494, -0.07452264, -0.00580554, -0.43584195,
       -0.01484818,  0.07201894,  0.00302151, -0.00906728,  0.05718661,
       -0.30969372,  0.1085849 ,  0.04075703, -0.12547907,  0.07853312,
        0.00103705,  0.02771755, -0.05737198,  0.04099937, -0.01

In [39]:
ranks = []
second_ranks = []
for doc_id in range(len(train_corpus)):
    inferred_vector = model.infer_vector(train_corpus[doc_id].words)
    sims = model.docvecs.most_similar([inferred_vector], topn=len(model.docvecs))
    rank = [docid for docid, sim in sims].index(doc_id)
    ranks.append(rank)
    
    second_ranks.append(sims[1])

/Users/HermanCheung/anaconda/envs/py36/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [40]:
collections.Counter(ranks)  # Results vary between runs due to random seeding and very small corpus

Counter({0: 4491,
         1: 107,
         6: 1,
         22: 1,
         61: 1,
         4: 4,
         7: 1,
         8: 2,
         2: 9,
         3: 4,
         21: 1,
         79: 1,
         207: 1,
         833: 1,
         10: 2,
         20: 1,
         63: 1,
         17: 2,
         13: 1,
         5: 2,
         23: 1,
         27: 1,
         16: 1,
         11: 1,
         57: 1,
         34: 1,
         15: 1})

In [46]:
import gensim
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.test.utils import get_tmpfile
fname=get_tmpfile("/Users/HermanCheung/Downloads/apnews_dbow/doc2vec.bin")
fname2=get_tmpfile("/Users/HermanCheung/Downloads/enwiki_dbow/doc2vec.bin")
from scipy import spatial
import smart_open
import pandas as pd

In [47]:
model = Doc2Vec.load(fname)
model2 = Doc2Vec.load(fname2)

/Users/HermanCheung/anaconda/envs/py36/lib/python3.6/site-packages/gensim/models/doc2vec.py:566: UserWarning: The parameter `iter` is deprecated, will be removed in 4.0.0, use `epochs` instead.
  warnings.warn("The parameter `iter` is deprecated, will be removed in 4.0.0, use `epochs` instead.")
/Users/HermanCheung/anaconda/envs/py36/lib/python3.6/site-packages/gensim/models/doc2vec.py:570: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")


In [53]:
file=pd.read_csv('Competition_Train_Data.txt', sep='\t')
file['m1_processed_s1']=file ['#1 String'].apply(lambda x : gensim.utils.simple_preprocess(x))
file['m1_processed_s2']=file ['#2 String'].apply(lambda x : gensim.utils.simple_preprocess(x))
file['m1_vector_distance_s1'] = file['m1_processed_s1'].apply (lambda x : model.infer_vector(x))
file['m1_vector_distance_s2'] = file['m1_processed_s2'].apply (lambda x : model.infer_vector(x))

In [54]:
file['m1_spartial_distance_s1_s2'] = file.apply(lambda row: spatial.distance.cosine(row['m1_vector_distance_s1'], row['m1_vector_distance_s2']), axis=1)

In [55]:

file['m2_processed_s1']=file ['#1 String'].apply(lambda x : gensim.utils.simple_preprocess(x))
file['m2_processed_s2']=file ['#2 String'].apply(lambda x : gensim.utils.simple_preprocess(x))
file['m2_vector_distance_s1'] = file['m2_processed_s1'].apply (lambda x : model2.infer_vector(x))
file['m2_vector_distance_s2'] = file['m2_processed_s2'].apply (lambda x : model2.infer_vector(x))

In [56]:
file['m2_spartial_distance_s1_s2'] = file.apply(lambda row: spatial.distance.cosine(row['m2_vector_distance_s1'], row['m2_vector_distance_s2']), axis=1)

In [57]:
file


,Label,Unique ID,#1 String,#2 String,m1_processed_s1,m1_processed_s2,m1_vector_distance_s1,m1_vector_distance_s2,m1_spartial_distance_s1_s2,m2_processed_s1,m2_processed_s2,m2_vector_distance_s1,m2_vector_distance_s2,m2_spartial_distance_s1_s2
0,1,494436,"Paul Themba Nyathi, an MDC spokesman, said the...",MDC spokesperson Paul Themba Nyathi said the A...,"[paul, themba, nyathi, an, mdc, spokesman, sai...","[mdc, spokesperson, paul, themba, nyathi, said...","[-0.07559167, 0.033623572, 0.01838631, -0.1238...","[-0.03182564, -0.018479582, 0.026145672, -0.16...",0.123538,"[paul, themba, nyathi, an, mdc, spokesman, sai...","[mdc, spokesperson, paul, themba, nyathi, said...","[0.13447003, -0.002164634, 0.08202701, -0.2240...","[0.22027768, 0.09206191, 0.12224323, -0.233031...",0.068368
1,1,659182,Pappas said he wouldn't hesitate about asking ...,"Pappas, the teacher, said he wouldn't hesitate...","[pappas, said, he, wouldn, hesitate, about, as...","[pappas, the, teacher, said, he, wouldn, hesit...","[0.13249256, 0.04294431, -0.2213545, 0.0002965...","[0.09912777, 0.01679686, -0.17728765, 0.018660...",0.069180,"[pappas, said, he, wouldn, hesitate, about, as...","[pappas, the, teacher, said, he, wouldn, hesit...","[0.08196573, -0.054595396, -0.019234974, -0.10...","[0.060406238, -0.028481139, -0.014025547, 0.01...",0.072411
2,1,400946,MSN Messenger 6 will be available for download...,The MSN Messenger 6 software will be available...,"[msn, messenger, will, be, available, for, dow...","[the, msn, messenger, software, will, be, avai...","[0.22164679, 0.24448867, -0.08086546, 0.002527...","[0.17652684, 0.30907086, -0.091411784, 0.12301...",0.225148,"[msn, messenger, will, be, available, for, dow...","[the, msn, messenger, software, will, be, avai...","[0.24972168, 0.12381417, 0.008083175, -0.01105...","[0.18536434, 0.046948854, 0.029670052, -0.0532...",0.125029
3,1,812777,The lies and deceptions from Saddam have been ...,It has been well documented over 12 years of l...,"[the, lies, and, deceptions, from, saddam, hav...","[it, has, been, well, documented, over, years,...","[-0.1730519, 0.036301848, 0.089646734, 0.00489...","[-0.08125144, -0.029402277, 0.06998825, -0.025...",0.128252,"[the, lies, and, deceptions, from, saddam, hav...","[it, has, been, well, documented, over, years,...","[0.0901071, -0.08832584, -0.035157416, -0.0347...","[0.065312214, -0.021889389, -0.019297782, -0.1...",0.089704
4,1,401674,Delta personnel at Atlanta's Hartsfield-Jackso...,Delta personnel at Hartsfield-Jackson Internat...,"[delta, personnel, at, atlanta, hartsfield, ja...","[delta, personnel, at, hartsfield, jackson, in...","[0.11185334, 0.18607965, 0.013748435, -0.12346...","[0.1853718, 0.13973594, -0.017687632, -0.08430...",0.127946,"[delta, personnel, at, atlanta, hartsfield, ja...","[delta, personnel, at, hartsfield, jackson, in...","[0.1526043, 0.03009424, -0.1506366, -0.0133250...","[0.30170557, 0.09310548, -0.015367906, -0.0750...",0.187593
5,1,10967,Hormone replacement therapy that combines estr...,A major British study has added to the evidenc...,"[hormone, replacement, therapy, that, combines...","[major, british, study, has, added, to, the, e...","[0.090661205, -0.03716793, -0.056047685, -0.13...","[0.16122156, 0.07900545, 9.6696094e-05, -0.131...",0.193631,"[hormone, replacement, therapy, that, combines...","[major, british, study, has, added, to, the, e...","[0.19004753, 0.1326356, -0.17454049, 0.0620978...","[0.2735636, 0.014351341, -0.18444666, -0.11024...",0.166649
6,0,597218,Kodak said the PracticeWorks deal is expected ...,The PracticeWorks acquisition is expected to a...,"[kodak, said, the, practiceworks, deal, is, ex...","[the, practiceworks, acquisition, is, expected...","[0.012721595, -0.06261634, 0.015895443, 0.1664...","[0.026854554, -0.08952692, 0.051034108, 0.2935...",0.244107,"[kodak, said, the, practiceworks, deal, is, ex...","[the, practiceworks, acquisition, is, expected...","[0.16923288, 0.1278077, -0.0057036257, -0.0538...",

NameError: name 'fname2' is not defined

In [14]:
string1 = gensim.utils.simple_preprocess("Paul Themba Nyathi, an MDC spokesman, said the Avenues Clinic was being targeted because it was treating opposition supporters.")
string2 = gensim.utils.simple_preprocess("MDC spokesperson Paul Themba Nyathi said the Avenues Clinic was targeted because it was handling opposition supporters.")



['paul', 'themba', 'nyathi', 'an', 'mdc', 'spokesman', 'said', 'the', 'avenues', 'clinic', 'was', 'being', 'targeted', 'because', 'it', 'was', 'treating', 'opposition', 'supporters']


In [16]:
v1 = model.infer_vector(string1)
v2 = model.infer_vector(string2)

In [20]:
spatial.distance.cosine(v1, v2)

0.1328372359275818